In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
f = open('./openai_api.txt')
api_key = f.read()

In [ ]:
chat = ChatOpenAI(openai_api_key=api_key, temperature=0.0, model="gpt-3.5-turbo")

In [ ]:
def generate_sql_from_nlp(nlp_query):
    # System prompt
    system_template = '''
    You're an expert DBA. You have a table called 'customer_reviews' with columns 'review_id' and 'review_text'.
    Column 'review_id' is numeric starting from 1.
    Users need to fetch 'review_text' for sentiment analysis.
    You need to write a SQL query to select the review text for the review_id given by the user.
    '''
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

    # Human prompt
    human_template = '''
    {nlp_query}
    '''
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    # Chat prompt
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    # Inject variables
    request = chat_prompt.format_prompt(nlp_query=nlp_query).to_messages()
    # Chat request
    result = chat(request)
    return result.content

In [ ]:
sql_query = generate_sql_from_nlp('show me seventh review')

In [ ]:
sql_query

In [ ]:
import snowflake.connector
# Snowflake connection details
conn_details = {
    "user": "llm_user",
    "password": "xxxxxxxx",
    "account": "xxxxx-xxxxx",
    "warehouse": "llm_wh",
    "database": "llm_db",
    "schema": "llm_schema"
}

def query_snowflake(sql_query):
    conn = snowflake.connector.connect(**conn_details)
    cur = conn.cursor()
    try:
        cur.execute(sql_query)
        result = cur.fetchone()
        return result
    finally:
        cur.close()
        conn.close()

review_text = query_snowflake(sql_query) 
print("Fetched Review Text:", review_text)

In [ ]:
def analyze_sentiment(review):
    # System prompt
    system_template = '''
    You need to analyze the sentiment of a given review.
    You also need to be very specific and respond ONLY and ONLY with one of the three options given below: 
    1. positive
    2. negative
    3. neutral
    '''
    human_template = """
    Analyze the sentiment for this review: {review}
    """
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # Inject variables
    request = chat_prompt.format_prompt(review=review).to_messages()
    # Chat request
    result = chat(request)
    return result.content

In [ ]:
sentiment = analyze_sentiment(review_text)
print(sentiment)